In [0]:
"""
Find CEO Subordinate Hierarchy salary difference (Hint: Use recursive CTE)

RECURSIVE CTE SYNTAX:

WITH RECURSIVE CTE AS (
    anchor / base query --> Executed only once in the beginning of the iteration

    UNION ALL

    recursive query --> executed from 2nd iteration onwards till the break condition is reached
    + break condition (like where)
)


Input
+-----------+-------------+----------+------+
|employee_id|employee_name|manager_id|salary|
+-----------+-------------+----------+------+
|          1|        Alice|      NULL|150000|
|          2|          Bob|         1|120000|
|          3|      Charlie|         1|110000|
|          4|        David|         2|105000|
|          5|          Eve|         2|100000|
|          6|        Frank|         3| 95000|
|          7|        Grace|         3| 98000|
|          8|        Helen|         5| 90000|
+-----------+-------------+----------+------+


Output
subordinate_id	subordinate_name	hierarchy_level	salary_difference
2	                Bob	                1	            -30000
3	                Charlie	            1	            -40000
4	                David	            2	            -45000
5	                Eve	                2	            -50000
6	                Frank	            2	            -55000
7	                Grace	            2	            -52000
8	                Helen	            3	            -60000
"""

'\nFind CEO Subordinate Hierarchy salary difference (Hint: Use recursive CTE)\n\nRECURSIVE CTE SYNTAX:\n\nWITH RECURSIVE CTE AS (\n    anchor / base query --> Executed only once in the beginning of the iteration\n\n    UNION ALL\n\n    recursive query --> executed from 2nd iteration onwards till the break condition is reached\n    + break condition (like where)\n)\n\n\nInput\n+-----------+-------------+----------+------+\n|employee_id|employee_name|manager_id|salary|\n+-----------+-------------+----------+------+\n|          1|        Alice|      NULL|150000|\n|          2|          Bob|         1|120000|\n|          3|      Charlie|         1|110000|\n|          4|        David|         2|105000|\n|          5|          Eve|         2|100000|\n|          6|        Frank|         3| 95000|\n|          7|        Grace|         3| 98000|\n|          8|        Helen|         5| 90000|\n+-----------+-------------+----------+------+\n\n\nOutput\nsubordinate_id\tsubordinate_name\thierarchy_l

In [0]:
%sql

DROP TABLE employees_ ;

CREATE TABLE IF NOT EXISTS employees_ (
    employee_id INT PRIMARY KEY,
    employee_name VARCHAR(50),
    manager_id INT,
    salary INT
);

-- Insert statements
INSERT INTO employees_ (employee_id, employee_name, manager_id, salary) VALUES
(1, 'Alice', NULL, 150000),
(2, 'Bob', 1, 120000),
(3, 'Charlie', 1, 110000),
(4, 'David', 2, 105000),
(5, 'Eve', 2, 100000),
(6, 'Frank', 3, 95000),
(7, 'Grace', 3, 98000),
(8, 'Helen', 5, 90000);

with recursive employee_hierarchy as (
    select employee_id, employee_name, manager_id, salary, 0 as hierarchy_level from employees_ where employee_id=1 -- select top level employee/CEO
    UNION ALL
    select t1.employee_id, t1.employee_name, t1.manager_id, t1.salary, hierarchy_level + 1 as hierarchy_level from employees_ t1
    inner join employee_hierarchy t2 on t1.manager_id=t2.employee_id
), manager_salary (
  select salary as boss_salary from employees_ where manager_id is null
)
select employee_id as subordinate_id, employee_name as subordinate_name, hierarchy_level, (salary - boss_salary) as salary_difference
from employee_hierarchy as t1 join manager_salary t2 on hierarchy_level != 0
order by hierarchy_level, subordinate_id



subordinate_id,subordinate_name,hierarchy_level,salary_difference
2,Bob,1,-30000
3,Charlie,1,-40000
4,David,2,-45000
5,Eve,2,-50000
6,Frank,2,-55000
7,Grace,2,-52000
8,Helen,3,-60000
